In [1]:
import sys
sys.path.append('../')

from src.datamodule.aenet_datamodule import AenetDataModule
from src.models.bnn import BNN, NN
from src.models.nets.network import NetAtom

ckpt_path = "/home/riccardo/bin/repos/aenet-bnn/src/results/PdO/pretrained/4/checkpoints/pretrained.ckpt"

datamodele = AenetDataModule(
    data_dir='../data/PdO/train.in',
    batch_size=200,
    test_split=0.1,
    valid_split=0.1
    )

net = NetAtom(
    datamodele.input_size, 
    datamodele.hidden_size, 
    datamodele.species, 
    datamodele.active_names, 
    datamodele.alpha,
    'cpu'
)

/home/riccardo/anaconda3/envs/bayesian/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torch.optim import Adam
model_kwargs = {'net': net,
        'lr': 0.00025616626859234823,
        'pretrain_epochs': 5,
        'mc_samples_train': 1,
        'mc_samples_eval': 20, 
        'dataset_size': datamodele.train_size, 
        'fit_context': 'lrt', 
        'prior_loc': 0, 
        'prior_scale': 0.3726682199695302, 
        'guide': 'normal', 
        'q_scale': 0.00127000766093029489207278289472795143,
        'obs_scale' :  0.8115512648735741}
model = BNN(**model_kwargs)

In [3]:
net = NN.load_from_checkpoint(
                ckpt_path,
                net=model.net,
            ).net
model.net = net

In [4]:
import pytorch_lightning as L

trainer = L.Trainer(max_epochs = 500)
trainer.fit(model=model, datamodule=datamodele)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/riccardo/anaconda3/envs/bayesian/lib/python3.12/site-packages/torch/utils/data/dataset.py:449: UserWarning: Length of split at index 0 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/riccardo/anaconda3/envs/bayesian/lib/python3.12/site-packages/torch/utils/data/dataset.py:449: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/riccardo/anaconda3/envs/bayesian/lib/python3.12/site-packages/torch/utils/data/dataset.py:449: UserWarning: Length of split at index 2 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/riccardo/anaconda3/envs/bayesian/lib/python3.12/site-packages/pytorch_lightning/core/optimizer.py:181: `LightningModule.conf

Epoch 0:   8%|▊         | 5/61 [00:00<00:02, 20.65it/s, v_num=46]          

/home/riccardo/anaconda3/envs/bayesian/lib/python3.12/site-packages/tyxe/likelihoods.py:260: UserWarning: var(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1760.)
  scale = predictions.var(dim).add(self.scale ** 2).sqrt()
/home/riccardo/anaconda3/envs/bayesian/lib/python3.12/site-packages/pytorch_lightning/loops/optimization/automatic.py:132: `training_step` returned `None`. If this was on purpose, ignore this warning...


Epoch 54:   5%|▍         | 3/61 [00:01<00:29,  1.96it/s, v_num=46] 